In [1]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.filters import threshold_li  # Changed from threshold_otsu to threshold_li
from skimage.measure import label
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x40/Golgi/tophat'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Golgi'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # Extract Golgi channel based on image format
    if len(img.shape) == 2:
        # Single channel image (already Golgi)
        print("  Detected single-channel image, using as is")
        golgi_channel = img
    elif len(img.shape) == 3 and img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        golgi_channel = img[2]  # Third channel
    elif len(img.shape) == 3 and img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        golgi_channel = img[:, :, 2]  # Third channel
    else:
        print(f"  Unexpected image shape: {img.shape}. Using first channel/plane.")
        if len(img.shape) == 3:
            golgi_channel = img[0] if img.shape[0] < img.shape[1] else img[:, :, 0]
        else:
            golgi_channel = img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    golgi_smoothed = gaussian(golgi_channel, sigma=1)

    # Apply Li thresholding
    print("  Performing Li thresholding...")
    thresh = threshold_li(golgi_smoothed)
    thresh = 0.9 * thresh  # Decrease threshold to involve more pixels
    binary = golgi_smoothed > thresh

    # Label connected components
    print("  Labeling connected components...")
    labels = label(binary)  # Directly label the binary image without opening/closing
    print(f"  Segmentation complete. Found {len(np.unique(labels))-1} objects")

    # Save the labeled image (main output)
    labels_output = os.path.join(output_dir, f"{base_name}_golgi_labels.tif")
    tifffile.imwrite(labels_output, labels.astype(np.uint32))
    print(f"  Saved labeled image to {labels_output}")

    # Print some statistics about the segmentation
    print(f"  Threshold value used: {thresh}")
    print(f"  Found {len(np.unique(labels))-1} Golgi objects")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Mounted at /content/drive
Found 13 .tif files to process
Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Golgi_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Performing Li thresholding...
  Labeling connected components...
  Segmentation complete. Found 493 objects
  Saved labeled image to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Golgi/denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Golgi_regional_tophat_golgi_labels.tif
  Threshold value used: 0.055099516544112534
  Found 493 Golgi objects
  Processing complete for denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Golgi_regional_tophat.tif

Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq002_Golgi_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Performing Li thresholding...
  Labeling connected components...
  Segmentation complete.